# Japanese model

In [22]:
#Torch
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, DataCollatorForLanguageModeling, TrainerCallback

#Datasets
from datasets import load_dataset

#Trainer
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model, PeftModel
from transformers import TrainingArguments, DataCollatorForLanguageModeling

## Model

In [23]:
tokenizer = AutoTokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b", use_fast=False)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained("rinna/bilingual-gpt-neox-4b", quantization_config=bnb_config)

if torch.cuda.is_available():
    model = model.to("cuda")
    print("CUDA is available! GPU is ready to use.")
    print("GPU Device:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")

Loading weights: 100%|██████████| 436/436 [00:11<00:00, 39.56it/s, Materializing param=gpt_neox.layers.35.post_attention_layernorm.weight]  


CUDA is available! GPU is ready to use.
GPU Device: AMD Radeon RX 7800 XT


In [24]:
ds = load_dataset("NilanE/ParallelFiction-Ja_En-100k")
split = ds['train'].train_test_split(test_size=0.01, seed=42)
ds = split["test"]

In [34]:
#training
def format_instruction(sample):

    genres_list = sample['meta']['novelupdates']['genres']
    #Example: "Adventure, Fantasy, Shounen"
    genres_str = ", ".join(genres_list)
    #We ask the model to write a story including these specific genres.
    instruction = f"「{genres_str}」のジャンルが含まれるストーリーを書いて。"
    #Standard Rinna Format: ユーザー: {instruction}\nシステム: {response}
    prompt = f"ユーザー: {instruction}\nシステム: {sample['src']}{tokenizer.eos_token}"
    
    return prompt

#sample
def format_instruction_sample(sample):
    genres_list = sample['meta']['novelupdates']['genres']
    genres_str = ", ".join(genres_list)
    instruction = f"「{genres_str}」のジャンルが含まれるストーリーを書いて。"
    prompt = f"ユーザー: {instruction}\nシステム: {sample['src']}{tokenizer.eos_token}"
    return {"text": prompt}

#Apply to the dataset
formatted_ds = ds.map(format_instruction_sample)

Map: 100%|██████████| 1061/1061 [00:01<00:00, 863.53 examples/s]


In [35]:
formatted_ds['text'][:3]

['ユーザー: 「Romance, Slice of Life」のジャンルが含まれるストーリーを書いて。\nシステム: 真面目な話をこれからすると思うと、自然と真面目な顔つきになる。\n姫川もさっきから、俺の事を直視しながら、どう切り出すか考えているのだろう。\n部屋には時計の秒針がコチコチ音を鳴らすだけで、他の音は聞こえなかった。\n静寂の中、ついに意を決し姫川は語り始める。\n「実は......」\n『ぐぅぅぅ~~~』\nこれは、姫川の作戦か? 俺は思わず頬を吊り上げ、にやけてしまった。\n「ひ、姫川。今の音は?」\n姫川は頬を赤くしながら下を向いてしまい、返事はない。\n少し肩が震えているので、恐らく恥ずかしさを我慢しているのだろう。\n「い、今の音は何でもありません。き、気にしないでください」\n「ちなみに、姫川が最後に食べたのはいつだ?」\n今だに頬を赤くしている姫川は、いつもより若干幼く見える。\n同い年だし、普段は俺よりもよっぽど大人っぽい仕草を見せられているが、今は後輩のように見える。\n「昨夜食べたっきりで、今日はまだ何も......」\n「そっか。話は後だ、ちょっと待ってろ」\n俺は、姫川をリビングに放置し台所に向かう。\n台所から手に持ったタウン誌を姫川に放り投げる。反射的に受け取った姫川はタウン誌片手に俺の方を見る。\n「今月のタウン誌。少し時間かかるからそれでも読んで暇つぶしててくれ。あ、そこの陰にコンセントあるから充電とか適当にしてていいぞ」\nそして俺は台所に行き、いつもつけている前掛けを装備する。\nそこまで料理は得意ではないが、それなりにできるはず。社長令嬢が食べていた食事と比べればお粗末なものができるだろう。\nだが、何も食べないよりはましだ。\n昨日作っておいた調理済みの料理を冷蔵庫から出し、皿に盛りつける。\nあとはご飯と、汁物。汁物は作り置きが無いので適当に作る。\n火の通りやすい野菜にワカメでいいか。\nそして、最後に魔法をかける。おそらく誰しもが使った事のある魔法。そう、レンチンだ。\n盛ったおかずに、ご飯をレンジに入れチンする。\n汁物だけは今作ったので温かさそのまま。\n――チン!\nレンジから取り出した料理をトレイにおき、箸をそえ台所を後にする。\n俺は雑誌を読んでる姫川の目の前に、適当に準備したご飯を置いていく。

In [36]:
split_ds = formatted_ds.train_test_split(test_size=0.1, seed=10) #seeded so the split is not random

train_set = split_ds['train']
eval_set = split_ds['test']

In [37]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [38]:
print(model)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(65536, 2816)
    (emb_dropout): Dropout(p=0.1, inplace=False)
    (layers): ModuleList(
      (0-35): 36 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2816,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2816,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.1, inplace=False)
        (post_mlp_dropout): Dropout(p=0.1, inplace=False)
        (attention): GPTNeoXAttention(
          (query_key_value): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2816, out_features=8448, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2816, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=8448, bi

## Training

### AMD GPU (set to 16GB of VRAM with ROCM)

In [39]:
#SFTConfig
sft_config = SFTConfig(
    output_dir="./lora-fine-tuned-model",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    logging_steps=10,
    save_strategy="epoch",
    
    #SFT Specific Params
    max_length=1024,
    dataset_text_field="text",
    packing=False,
    
    #AMD/Performance Settings
    bf16=True,
    #Explicitly False for AMD compatibility
    tf32=False,
    gradient_checkpointing=True,
    
    #Optimizer
    optim="adamw_torch",               
    dataloader_pin_memory=True,
    group_by_length=True
)

#Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_set,
    eval_dataset=eval_set,
    peft_config=lora_config,
    processing_class=tokenizer,
    args=sft_config,
    formatting_func=format_instruction
)

trainer.train()

/home/megad/.local/lib/python3.12/site-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/home/megad/.local/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Truncating eval dataset: 100%|██████████| 107/107 [00:00<00:00, 13544.71 examples/s]
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 0}.
/home/megad/.local/lib/python3.12/site-packages/transformers/integrations/sdpa_attention.py:92: UserWarning: Flash Efficient attention on Current AMD GP

KeyboardInterrupt: 

### NVIDIA GPU (set to 8GB of VRAM with cuda)

In [ ]:
sft_config = SFTConfig(
    output_dir="./lora-fine-tuned-model",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    logging_steps=10,
    save_strategy="epoch",
    
    #Sequence Length Optimization
    max_length=512,
    packing=False,
    
    #General GPU Optimizations (NVIDIA)
    fp16=True,
    #Set to True only if on 30/40-series                        
    bf16=False,
    gradient_checkpointing=True,
    
    # Optimizer
    optim="paged_adamw_32bit",
    dataloader_pin_memory=True
)

#Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_set,
    eval_dataset=eval_set,
    peft_config=lora_config,
    processing_class=tokenizer,
    args=sft_config,
    formatting_func=format_instruction
)

#Execute
trainer.train()

/home/megad/.local/lib/python3.12/site-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/home/megad/.local/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Truncating eval dataset: 100%|██████████| 107/107 [00:00<00:00, 2910.46 examples/s]
Exception ignored in: <function Dataset.__del__ at 0x76bb8ba307c0>
Traceback (most recent call last):
  File "/home/megad/.local/lib/python3.12/site-packages/datasets/arrow_dataset.py", line 1497, in __del__
    def __del__(self):

KeyboardInterrupt: 


NotImplementedError: "_amp_foreach_non_finite_check_and_unscale_cuda" not implemented for 'BFloat16'

# Evaluation

In [41]:
# Note:
# Cap at 512 or 1024 tokens next time instead of whole chapters cause this took way too long 
lora_checkpoint_path = "./lora-fine-tuned-model/checkpoint-477"
model = PeftModel.from_pretrained(model, lora_checkpoint_path)
model = model.merge_and_unload()

/home/megad/.local/lib/python3.12/site-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [42]:
#One prompt to test
text = "「Shounen, Adventure」のジャンルが含まれるストーリーを書いて。"
token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=100,
        min_new_tokens=100,
        do_sample=True,
        temperature=1.0,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

「Shounen, Adventure」のジャンルが含まれるストーリーを書いて。 また、あなたはあなたの小説を他の人が読んでどのように思うか尋ねます。 あなたがそれを聞いたら 「私は他の人がどのように感じるか知りたい」 「私が書いた小説を他の人がどのようにして読むか知りません。 私は私の作品の他の作家に読んでもらいたいです、それは私をさらに悪化させます。 私の小説は、あなたが他の人にそれを渡すまで、あなたの小説よりも優れているでしょうか? 」 「あなたはあなたの小説を他の人にどのように知ってもらいたいですか? 


In [ ]:

target_length = 2000
max_new_tokens_per_loop = 512 
story = "「Shounen, Adventure」のジャンルが含まれるストーリーを書いて。"

#Encode the initial prompt
input_ids = tokenizer.encode(story, return_tensors="pt").to("cuda")

print("--- Starting the Story ---")
print(story, end="")

while input_ids.shape[1] < target_length:
    #Generate the next segment
    output_sequences = model.generate(
        input_ids=input_ids,
        max_new_tokens=max_new_tokens_per_loop,
        do_sample=True,
        top_p=0.95,
        temperature=0.8,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id
    )

    new_tokens = output_sequences[:, input_ids.shape[1]:]
    
    new_text = tokenizer.decode(new_tokens[0], skip_special_tokens=True)
    print(new_text, end="", flush=True)

    input_ids = output_sequences

    # Safety break if the model generates an eos token
    if tokenizer.eos_token_id in new_tokens:
        print("\n--- Model reached end of story. ---")
        break

--- Starting the Story ---
「Shounen, Adventure」のジャンルが含まれるストーリーを書いて。第2回では、Webアプリケーションの開発におけるオブジェクト指向設計について解説しました。
前回の記事はこちらからお読みください 第1回 オブジェクト指向設計の基礎知識~3つの基本概念とオブジェクト指向プログラミング~ オブジェクト指向の設計手法として、まず最初に覚えておきたいのが、クラスの階層です。クラスというのは、ある処理をするために必要な ...
今回は、オブジェクト指向設計について紹介します。 オブジェクト指向設計とは、オブジェクト指向プログラミング言語で記述されたソフトウェア(あるいは、その開発環境)を設計することを指します。 オブジェクト指向言語のプログラムでは、クラスというオブジェクト ...
オブジェクト指向の基本となる考え方の1つに、「カプセル化」があります。 これは、データや処理を隠蔽して実装を分離することです。たとえば、以下の図は、Javaによるオブジェクト指向プログラミングの例です。この図において、処理中のデータは、カプセル化さ ...
--- Model reached end of story. ---
